# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Creation DataFrame

Use pip to install folium, otherwise the import gives an error statement

In [1]:
!pip install folium

    100% |████████████████████████████████| 81kB 3.0MB/s ta 0:00:01
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt

In [2]:
#Import necessary libraries:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Part I:

Scrape the Wikipedia page:

link for Postal Codes: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

I found the following method of web scraping here: https://stackoverflow.com/questions/39710903/pd-read-html-imports-a-list-rather-than-a-dataframe

In [3]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')  #Reads the wiki table into a pd_DataFrame directly!!
df = dfs[0]
array1 = df.values  #Create a numpy array to be able to work with the data more effectively

To check the result:

In [4]:
print(array1)

[['M1ANot assigned' 'M2ANot assigned' 'M3ANorth York(Parkwoods)'
  'M4ANorth York(Victoria Village)'
  'M5ADowntown Toronto(Regent Park / Harbourfront)'
  'M6ANorth York(Lawrence Manor / Lawrence Heights)'
  "M7AQueen's Park Ontario Provincial Government" 'M8ANot assigned'
  'M9AEtobicoke(Islington Avenue)']
 ['M1BScarborough(Malvern / Rouge)' 'M2BNot assigned'
  'M3BNorth York(Don Mills)North'
  'M4BEast York(Parkview Hill / Woodbine Gardens)'
  'M5BDowntown Toronto(Garden District, Ryerson)'
  'M6BNorth York(Glencairn)' 'M7BNot assigned' 'M8BNot assigned'
  'M9BEtobicoke(West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale)']
 ['M1CScarborough(Rouge Hill / Port Union / Highland Creek)'
  'M2CNot assigned' 'M3CNorth York(Don Mills)South(Flemingdon Park)'
  'M4CEast York(Woodbine Heights)' 'M5CDowntown Toronto(St. James Town)'
  'M6CYork(Humewood-Cedarvale)' 'M7CNot assigned' 'M8CNot assigned'
  'M9CEtobicoke(Eringate / Bloordale Gardens / Old Burnhamthorpe / Mark

Now, I will create the lists with the postal codes, the boroughs, and neighborhoods that will then be used to create the final product, the dataframe:

In [5]:
#Create a list of postal codes:
print('there are',len(array1[0]), 'columns in the numpy array, which reflects the nine columns of the Wikipedia page')
list_codes = []
for col in array1:
    for element in col:
        if 'Not assigned' not in element:
            list_codes += [element[:3]]

#Create a list of boroughs:
list_boroughs = []
for col in array1:
    for element in col:
        if 'Not assigned' not in element:
            index_end = element.find('(')
            list_boroughs += [element[3:index_end]]
            
print(list_codes)
print(list_boroughs) #From the print statement we can notice that there are a few misspellings
#Correct the misspellings:
list_boroughs[-3] = 'East Toronto Business reply mail Processing Centre 969 Eastern'
list_boroughs[4] = "Queen's Park Ontario Provincial Government"
list_boroughs[35] = "East YorkEast Toronto"
list_boroughs[-9] = 'Etobicoke Northwest'
list_boroughs[-11] = 'Downtown Toronto Stn A PO Boxes25 The Esplanade'
#Fixed:
print()
print()
print("The corrected list is the following:")
print(list_boroughs)

there are 9 columns in the numpy array, which reflects the nine columns of the Wikipedia page
['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B', 'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C', 'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H', 'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J', 'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L', 'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M', 'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N', 'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R', 'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S', 'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V', 'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X', 'M4Y', 'M7Y', 'M8Y', 'M8Z']
['North York', 'North York', 'Downtown Toronto', 'North York', "Queen's Park Ontario Provincial Governmen", 'Etobicoke', 'Scarborough', 'North York', 'East York', 'Downtown Toronto', '

In [19]:
#Finally, let's retrieve the neighborhoods
list_neigh_temp = []
for col in array1:
    for element in col:
        if 'Not assigned' not in element:
            index_start = element.find('(') +1
            list_neigh_temp += [element[index_start:-1]]
list_neigh_temp[4] = "Queen's Park"  #typo
print(list_neigh_temp)

list_neigh = []
for string in list_neigh_temp:
    temporary = string.replace(" /",",")
    list_neigh += [temporary]
#Change the name of a few mistyped element:
list_neigh[7] = "Don Mills"
list_neigh[13] = "Don Mills, South, Flemingdon Park"
list_neigh[40] = "Downsview, East, CFB Toronto" 
list_neigh[53] = "Downsview, Central" 
list_neigh[59] = "Willowdale, South"
list_neigh[60] = "Downsview, Northwest"
list_neigh[72] = "Willowdale, West"
print()
print(list_neigh)

['Parkwoods', 'Victoria Village', 'Regent Park / Harbourfront', 'Lawrence Manor / Lawrence Heights', "Queen's Park", 'Islington Avenue', 'Malvern / Rouge', 'Don Mills)Nort', 'Parkview Hill / Woodbine Gardens', 'Garden District, Ryerson', 'Glencairn', 'West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale', 'Rouge Hill / Port Union / Highland Creek', 'Don Mills)South(Flemingdon Park', 'Woodbine Heights', 'St. James Town', 'Humewood-Cedarvale', 'Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood', 'Guildwood / Morningside / West Hill', 'The Beaches', 'Berczy Park', 'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street', 'Christie', 'Cedarbrae', 'Hillcrest Village', 'Bathurst Manor / Wilson Heights / Downsview North', 'Thorncliffe Park', 'Richmond / Adelaide / King', 'Dufferin / Dovercourt Village', 'Scarborough Village', 'Fairview / Henry Farm / Oriole', 'Northwood Park / York University', 'The Danforth East', 'Harbourfront East / Union Statio

The final step is creating the dataframe with, as columns, the three lists precedently created:

In [20]:
df_pc = pd.DataFrame(data = { 'PostalCode': list_codes,'Borough':list_boroughs,'Neighborhood':list_neigh})
df_pc = df_pc[['PostalCode','Borough','Neighborhood']]
df_pc#.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park Ontario Provincial Government,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [21]:
print("The dataframe has the following dimensions:", df_pc.shape)

The dataframe has the following dimensions: (103, 3)


## END PART 1

## Part 2: Latitude and Longitude

In [22]:
# The code was removed by DSX for sharing.

In [23]:
# construct URL to make API call
url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, 'toronto, M5G')

response = requests.get(url).json() # get response
print('Test:')
print(response)
geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
latitude = geographical_data['lat']
longitude = geographical_data['lng']
print(latitude,longitude)

Test:
{'results': [{'formatted_address': 'Toronto, ON M5G, Canada', 'types': ['postal_code', 'postal_code_prefix'], 'place_id': 'ChIJQ4yqgck0K4gR1f-j38zDpt0', 'geometry': {'location': {'lat': 43.6579524, 'lng': -79.3873826}, 'viewport': {'northeast': {'lat': 43.662463, 'lng': -79.380601}, 'southwest': {'lat': 43.6511301, 'lng': -79.391719}}, 'location_type': 'APPROXIMATE', 'bounds': {'northeast': {'lat': 43.662463, 'lng': -79.380601}, 'southwest': {'lat': 43.6511301, 'lng': -79.391719}}}, 'address_components': [{'long_name': 'M5G', 'types': ['postal_code', 'postal_code_prefix'], 'short_name': 'M5G'}, {'long_name': 'Old Toronto', 'types': ['political', 'sublocality', 'sublocality_level_1'], 'short_name': 'Old Toronto'}, {'long_name': 'Toronto', 'types': ['locality', 'political'], 'short_name': 'Toronto'}, {'long_name': 'Toronto Division', 'types': ['administrative_area_level_2', 'political'], 'short_name': 'Toronto Division'}, {'long_name': 'Ontario', 'types': ['administrative_area_leve

In [16]:
lat_list = []
long_list = []
for code in list_codes:
    url1 = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, 'toronto, ' + code)
    response_loop = requests.get(url1).json()
    geo_data = response_loop['results'][0]['geometry']['location']
    lat_list += [geo_data['lat']]
    long_list += [geo_data['lng']]
print('done')

done


In [17]:
print('length list latitudes: ',len(lat_list))
print('length list longitudes: ',len(long_list))

length list latitudes:  103
length list longitudes:  103


In [18]:
df_pc['Latitude'] = lat_list
df_pc['Longitude'] = long_list
df_pc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park Ontario Provincial Government,Queen's Park,43.662301,-79.389494


## END PART 2